In [1]:
import json, nltk, re, os, codecs

import pandas as pd
from pandas.io.json import json_normalize
#import pandas_profiling
import numpy as np

from matplotlib import pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_style("whitegrid")
import altair as alt
alt.renderers.enable("notebook")

# Code for hiding seaborn warnings
import warnings
warnings.filterwarnings("ignore")

from sklearn import feature_extraction

from pymongo import MongoClient

In [2]:

# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import re
import pickle
import nltk

nltk.download('punkt')
nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer


from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from nltk.tokenize import word_tokenize
import warnings

warnings.simplefilter('ignore')

import pickle
from sklearn.externals import joblib

[nltk_data] Downloading package punkt to /home/kells/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/kells/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
def load_data_from_momgodb(db_obj):
    dbname = db_obj['db']
    ip = db_obj['ip']
    port = db_obj['port']
    coll = db_obj['coll']
    connection = MongoClient(ip,port)
    db = connection[dbname]
    exclude_data = {'_id': False}
    raw_data = list(db[coll].find({}, projection=exclude_data))
    dataset = pd.DataFrame(raw_data)
    print(f'Data loaded from mongodb {coll} collection succesfully')
    return dataset

In [6]:
db_data = {
    'ip' :'10.10.250.0',
    'port' : 27017,
    'db' : 'projectfinder',
    'coll' : 'mldata1'
}

In [7]:
ready_data = load_data_from_momgodb(db_data)
ready_data.head()

Data loaded from mongodb mldata1 collection succesfully


,Data-Engr-Big Data,Data-Sci-BI,Dev-Devops,Dev-Web-Backend,Dev-Web-Frontend,Dev-Web-Fullstack,ERP-SAP,IT-Admin-Others,IT-Mgmt-Consulting,IT-Mgmt-Projectleiter,IT-Technical-Dev,IT/Elektrotechn,Infr-Admin-Database,Infr-Admin-Linux,Infr-Admin-Microsoft,Infr-Admin-Net,Infr-Database-Admin,SW-Dev-Mobile,SW-Dev-Others,project
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,Für einen unserer Kunden aus dem Finanzdienstl...
1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Business Intelligence Analyst (m/w) - Tableau ...
2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Konzeption, Customizing sowie Softwareanpassun..."
3,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,Future Consulting GmbH \r\nentscheiden Sie sic...
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,"Als Mitglied eines kleinen, dynamischen Teams ..."


In [12]:
#load saved model
import pickle
from sklearn.externals import joblib
filename = '../stopwords.sav'
stopwords = joblib.load(filename)

In [9]:
def tokenize(text):
    """Normalize, tokenize and stem text string
    
    Args:
    text: string. String containing message for processing
       
    Returns:
    stemmed: list of strings. List containing normalized and stemmed word tokens
    """
    # Stem word tokens and remove stop words
    stemmer_eng = SnowballStemmer("english", ignore_stopwords=True)
    stemmer_germ = SnowballStemmer("german", ignore_stopwords=True) 
    try:
        # Convert text to lowercase and remove punctuation
        text = re.sub("[^a-zA-Z ]", " ", text.lower()) #remove non alphbetic text

        # Tokenize words
        tokens = word_tokenize(text)
        stemmed = [stemmer_germ.stem(word) for word in tokens if word not in stopwords]
        stemmed = [stemmer_eng.stem(word) for word in stemmed if len(word) > 1]
        stemmed = [word for word in stemmed if word not in stopwords]
    except IndexError:
        pass

    return stemmed

In [14]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

In [15]:
tokenize('entwickler entwicklung bin ich i love running i am a jumped')

['entwickl', 'run', 'jump']

In [16]:
feature = ready_data['project']
target = ready_data.drop('project', axis=1)
target_name = list(target.columns.values)

In [18]:
#filename = 'models/feature.sav'
joblib.dump(feature, '../models/feature.sav')
joblib.dump(target, '../models/target.sav')

['../models/target.sav']

In [12]:
X_train, X_test, Y_train, Y_test = train_test_split(feature, target, random_state = 1)

np.random.seed(17)
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip..._score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=None))])

In [18]:
#save model

filename = 'model.pkl'
joblib.dump(pipeline, filename)

['model.pkl']

In [13]:
def get_eval_metrics(actual, predicted, col_names):
    """Calculate evaluation metrics for ML model
    
    Args:
    actual: array. Array containing actual labels.
    predicted: array. Array containing predicted labels.
    col_names: list of strings. List containing names for each of the predicted fields.
       
    Returns:
    metrics_df: dataframe. Dataframe containing the accuracy, precision, recall 
    and f1 score for a given set of actual and predicted labels.
    """
    metrics = []
    
    # Calculate evaluation metrics for each set of labels
    for i in range(len(col_names)):
        accuracy = accuracy_score(actual[:, i], predicted[:, i])
        precision = precision_score(actual[:, i], predicted[:, i])
        recall = recall_score(actual[:, i], predicted[:, i])
        f1 = f1_score(actual[:, i], predicted[:, i])
        
        metrics.append([accuracy, precision, recall, f1])
    
    # Create dataframe containing metrics
    metrics = np.array(metrics)
    metrics_df = pd.DataFrame(data = metrics, index = col_names, columns = ['Accuracy', 'Precision', 'Recall', 'F1'])
      
    return metrics_df

In [14]:
# Calculate evaluation metrics for training set
Y_train_pred = pipeline.predict(X_train)

AttributeError: 'Series' object has no attribute 'columns'

In [16]:
col_names = list(target.columns.values)

print(get_eval_metrics(np.array(Y_train), Y_train_pred, col_names))

                      Accuracy  Precision    Recall        F1
Big Data-Data Engr    0.998280   0.985989  0.985989  0.985989
D_Science/Analysis    0.999812   1.000000  0.988411  0.994172
ERP-SAP               0.999247   0.996542  0.997002  0.996772
IT/Admin-Database     0.999167   0.995458  0.981343  0.988350
IT/Admin-Linux        0.999355   0.991332  0.989772  0.990551
IT/Admin-Microsoft    0.997446   0.987216  0.988479  0.987847
IT/Admin-Others       0.999382   0.997366  0.982699  0.989978
IT/Consulting-PM      0.997070   0.992850  0.992979  0.992914
IT/Technisches        0.999839   0.998542  0.992754  0.995640
SW/Dev-Devops         0.999731   0.993781  0.993781  0.993781
SW/Dev-Mobile         0.999839   1.000000  0.991429  0.995696
SW/Dev-Web            0.995968   0.983607  0.984848  0.984227
SW/Dev-Web-Backend    0.998333   0.968142  0.976786  0.972444
SW/Dev-Web-Fullstack  0.995403   0.978866  0.980043  0.979455
other                 0.999301   0.997472  0.991621  0.994538


In [19]:
# Calculate evaluation metrics for test set
Y_test_pred = pipeline.predict(X_test)

eval_metrics0 = get_eval_metrics(np.array(Y_test), Y_test_pred, col_names)
print(eval_metrics0)

                      Accuracy  Precision    Recall        F1
Big Data-Data Engr    0.995806   0.972067  0.956044  0.963989
D_Science/Analysis    0.999597   1.000000  0.974490  0.987080
ERP-SAP               0.997742   0.985396  0.995084  0.990217
IT/Admin-Database     0.997016   0.995927  0.933206  0.963547
IT/Admin-Linux        0.997419   0.997680  0.932755  0.964126
IT/Admin-Microsoft    0.993468   0.972778  0.962758  0.967742
IT/Admin-Others       0.998306   0.986150  0.956989  0.971351
IT/Consulting-PM      0.991694   0.983600  0.975407  0.979486
IT/Technisches        0.998871   0.987603  0.956000  0.971545
SW/Dev-Devops         0.999194   0.989583  0.976027  0.982759
SW/Dev-Mobile         0.999194   1.000000  0.951923  0.975369
SW/Dev-Web            0.993710   0.973042  0.976804  0.974920
SW/Dev-Web-Backend    0.995323   0.942197  0.895604  0.918310
SW/Dev-Web-Fullstack  0.988629   0.970845  0.929518  0.949733
other                 0.995968   0.997392  0.940959  0.968354


In [7]:
#load saved model
loaded_pipeline = joblib.load('../models/model.pkl')

In [19]:
#load feature and target data
loaded_feature = joblib.load('../models/feature.sav')
loaded_target = joblib.load('../models/target.sav')

In [ ]:
#Y_pred = loaded_pipeline.predict(X_train)

In [20]:
def get_category(text, model, labels):
    class_labels = model.predict([text])[0]
    classification_results = dict(zip(labels.columns, class_labels))
    return classification_results
    

In [21]:
text_data ="""Für unseren Kunden in Nordrhein-Westfalen suchen wir ab sofort eine/n Senior Berater SAP TM (m/w/d).


 


Ihre Aufgaben:


- Erarbeitung und Umsetzung von neuen Konzepten im Rahmen unseres internationalen Rolloutprojektes


- Customizing/Konfiguration in den Bereichen SAP TM


- Erstellung von Spezifikationen für Entwicklungen und Schnittstellen


- Vorbereitung und Durchführung von Tests und User Trainings, sowie Unterstützung im Rahmen der Hypercare


 


Ihre Qualifikation:


- Sehr gute Kenntnisse von SAP TM und der Integration in die angrenzende Module (SD, MM, FI/CO)


- Erfahrung in der Implementierung von SAP TM für Verlader mit LKW-, Schiffscontainer-, Luftfracht- und Bahn-Transporten (Outbound & Inbound)


- Vorzugsweise Erfahrungen im Rahmen der Gefahrguttransporte


- Expertise in komplexen Frachtkostenabwicklungen


- Schnittstelle / Kommunikation über EDI


- Erfahrungen in der Integration zu SAP GTS und LBN


""" #ready_data['project'][0]
#text_data = tokenize(text_data)
#text_data = ' '.join(text_data)
text_data

'Für unseren Kunden in Nordrhein-Westfalen suchen wir ab sofort eine/n Senior Berater SAP TM (m/w/d).\n\n\n \n\n\nIhre Aufgaben:\n\n\n- Erarbeitung und Umsetzung von neuen Konzepten im Rahmen unseres internationalen Rolloutprojektes\n\n\n- Customizing/Konfiguration in den Bereichen SAP TM\n\n\n- Erstellung von Spezifikationen für Entwicklungen und Schnittstellen\n\n\n- Vorbereitung und Durchführung von Tests und User Trainings, sowie Unterstützung im Rahmen der Hypercare\n\n\n \n\n\nIhre Qualifikation:\n\n\n- Sehr gute Kenntnisse von SAP TM und der Integration in die angrenzende Module (SD, MM, FI/CO)\n\n\n- Erfahrung in der Implementierung von SAP TM für Verlader mit LKW-, Schiffscontainer-, Luftfracht- und Bahn-Transporten (Outbound & Inbound)\n\n\n- Vorzugsweise Erfahrungen im Rahmen der Gefahrguttransporte\n\n\n- Expertise in komplexen Frachtkostenabwicklungen\n\n\n- Schnittstelle / Kommunikation über EDI\n\n\n- Erfahrungen in der Integration zu SAP GTS und LBN\n\n\n'

In [22]:
get_category(text_data, loaded_pipeline, loaded_target)

{'Big Data-Data Engr': 0,
 'D_Science/Analysis': 0,
 'ERP-SAP': 1,
 'IT/Admin-Database': 0,
 'IT/Admin-Linux': 0,
 'IT/Admin-Microsoft': 0,
 'IT/Admin-Others': 0,
 'IT/Consulting-PM': 0,
 'IT/Technisches': 0,
 'SW/Dev-Devops': 0,
 'SW/Dev-Mobile': 0,
 'SW/Dev-Web': 0,
 'SW/Dev-Web-Backend': 0,
 'SW/Dev-Web-Fullstack': 0,
 'other': 0}

In [49]:
class_labels = pipeline.predict([text_data])[0]

In [50]:
classification_results = dict(zip(target.columns, class_labels))

In [51]:
classification_results

{'Big Data-Data Engr': 0,
 'D_Science/Analysis': 0,
 'ERP-SAP': 0,
 'IT/Admin-Database': 0,
 'IT/Admin-Linux': 0,
 'IT/Admin-Microsoft': 0,
 'IT/Admin-Others': 0,
 'IT/Consulting-PM': 0,
 'IT/Technisches': 0,
 'SW/Dev-Devops': 0,
 'SW/Dev-Mobile': 0,
 'SW/Dev-Web': 0,
 'SW/Dev-Web-Backend': 0,
 'SW/Dev-Web-Fullstack': 0,
 'other': 0}

In [52]:
class_labels

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])